In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import iplot
from plotly.subplots import make_subplots

# Creating a test dataset to make the graph with
df = pd.read_csv("/content/all_stocks_5yr.csv")
df = df.loc[df["Name"] == "AAL", ["date", "open", "high", "low", "close", "volume"]]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1259 entries, 0 to 1258
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    1259 non-null   object 
 1   open    1259 non-null   float64
 2   high    1259 non-null   float64
 3   low     1259 non-null   float64
 4   close   1259 non-null   float64
 5   volume  1259 non-null   int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 68.9+ KB


In [ ]:
df["date"] = pd.to_datetime(df["date"])

In [ ]:
# We'll need to calculate the moving average and standard dev for our bollinger bands
WINDOW = 30
df["sma"] = df["close"].rolling(WINDOW).mean()
df["std"] = df["close"].rolling(WINDOW).std(ddof=0)
df

,date,open,high,low,close,volume,sma,std
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,NaN,NaN
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,NaN,NaN
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,NaN,NaN
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,NaN,NaN
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,NaN,NaN
...,...,...,...,...,...,...,...,...
1254,2018-02-01,54.00,54.64,53.59,53.88,3623078,54.177333,2.407474
1255,2018-02-02,53.49,53.99,52.03,52.10,5109361,54.199333,2.385273
1256,2018-02-05,51.99,52.39,49.75,49.76,6878284,54.127333,2.483563
1257,2018-02-06,49.32,51.50,48.79,51.18,6782480,54.073667,2.528863


In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

fig.add_trace(go.Candlestick(x=df["date"],
                            open=df["open"],
                             high=df["high"],
                             low=df["close"],
                             close=df["close"],
                             showlegend=False,
                             name="candlestick"),
                row=1, col=1)

fig.add_trace(go.Scatter(x=df["date"],
                         y=df["sma"] + (df["std"] * 2),
                         line_color="gray",
                         line={"dash":"dash"},
                         opacity=0.5,
                         name="upper band"),
              row=1,col=1)

fig.add_trace(go.Scatter(x=df["date"],
                         y=df["sma"] - (df["std"] * 2),
                         line_color="gray",
                         line={"dash":"dash"},
                         fill="tonexty",
                         opacity=0.5,
                         name="lower band"),
              row=1,col=1)


adjusted_axis = [date for date in pd.date_range(start=min(df["date"]), end=max(df["date"])) if date not in df["date"].values]

fig.update_xaxes(rangebreaks=[dict(values=adjusted_axis)])

fig.show()